# **응급상황 자동 인식 및 응급실 연계 서비스**
# **단계1 : 응급상황 음성 인식 및 요약**

## **0.미션**

단계 1에서는, 응급상황의 음성을 인식해서 텍스트로 변환하고, 변환된 텍스트를 다시 요약 및 핵심키워드 도출 작업을 수행합니다.  
이를 위해 사전학습된 모델을 API로 연결하여 활용합니다.

### (1) 미션1
* 음성인식 : STT(Speech-to-Text)
    * 사용 모델 : OpenAI의 **Whisper-1**
    * 제공받은 음성 파일과 새로 제작하는 5건 이상의 음성파일을 텍스트로 변환하고, 변환작업이 잘 되는지 확인해 봅시다.

### (2) 미션2
* 텍스트 요약 및 핵심 키워드 도출
    * 사용 모델 : OpenAI의 **GPT-3.5-turbo**
    * 내용 요약과 주요 키워드를 도출하도록
    프롬프트 입력과 출력을 구성하고 테스트 해 봅시다.

* [추가]응급실 현황 다운로드(이 데이터는 단계3에서 필요합니다.)



## **1.환경설정**

### (1) 경로 설정

구글 드라이브 연결

#### 1) 구글 드라이브 폴더 생성
* 새 폴더(project6_2)를 생성하고
* 제공 받은 파일을 업로드

#### 2) 구글 드라이브 연결

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
path = '/content/drive/MyDrive/project6_2/'

### (2) 라이브러리

#### 1) 필요한 라이브러리 설치

* requirements.txt 파일의 [경로 복사]를 한 후,
* 아래 경로에 붙여 넣기

In [ ]:
# 경로 : /content/drive/MyDrive/project6_2/requirements.txt
# 경로가 다른 경우 아래 코드의 경로 부분을 수정하세요.

!pip install -r /content/drive/MyDrive/project6_2/requirements.txt

#### 2) 라이브러리 로딩

In [ ]:
#필요한 라이브러리 설치 및 불러우기
import os
import requests
import xml.etree.ElementTree as ET
import pandas as pd
import matplotlib.pyplot as plt
import openai
from openai import OpenAI
import json

# 더 필요한 라이브러리 추가 -------------




### (3) OpenAI API Key 환경 변수 설정

* 제공받은 open ai api key를 **api_key.txt** 파일에 저장합니다.
    * (제공받은 api_key.txt 파일은 비어 있습니다.)

* 다음 코드를 통해 환경변수로 등록 합니다.

In [ ]:
def load_file(filepath):
    with open(filepath, 'r') as file:
        return file.readline().strip()

# API 키 로드 및 환경변수 설정
openai.api_key = load_file(path + 'api_key.txt')
os.environ['OPENAI_API_KEY'] = openai.api_key

* ⚠️ 아래 코드셀은, 실행해서 key가 제대로 보이는지 확인하고 결과는 삭제하세요.

In [ ]:
print(os.environ['OPENAI_API_KEY'])

## **2. 미션1 : STT**

### (1) 제공된 데이터 변환
* 세부사항
    * 사용 모델 : whisper-1
    * 제공 받은 오디오 파일을 읽어서 텍스트로 변환시켜 봅시다.
        * 반복문을 통해 파일 하나씩 읽어서 텍스트 변환
        * 변환된 텍스트를 데이터 프레임에 추가

|filename|text|
|----|----|
|audio3.mp3|어쩌구 저쩌구...급해요.|

* 음성파일 변환

In [ ]:
# 음성파일 경로 지정


In [ ]:
# OpenAI 클라이언트 생성
client =

In [ ]:
# 위스퍼 모델 사용 : 제공된 음성파일 중 1개를 텍스트로 변환해보기
filename = 'audio2.mp3'
audio_file = open(audio_path + filename, "rb")
transcript = client.audio.transcriptions.create(
    file=audio_file,
    model="whisper-1",
    language="ko",
    response_format="text",
)

print(transcript, type(transcript))

* 음성파일 변환 함수 생성

In [ ]:
def audio_to_text(audio_path, filename):
    # OpenAI 클라이언트 생성


    # 오디오 파일을 읽어서, 위스퍼를 사용한 변환


    # 결과 반환



In [ ]:
# 음성파일 이름을 리스트에 담기
file_names = [f for f in os.listdir(audio_path) if os.path.isfile(os.path.join(audio_path, f))]
print(file_names)

In [ ]:
# 반복문을 통해, 파일 하나씩 읽어서 텍스트 변환, 변환된 텍스트를 데이터 프레임에 추가

# 빈 데이터프레임 선언


# 반복문 수행하면서 오디오 변환


# 데이터프레임 결과 조회


### (2) 오디오 데이터 추가 수집(제작) 및 변환

* 세부사항
    * 응급 상황에 맞는 음성 녹음하기
        * 응급 등급별 1개 이상씩(총 5개 이상)
    * 반복문을 통해 모든 음성 파일 데이터 변환 : STT
        * 변환 내용은 위에서 저장한 데이터프레임에 추가
    * 변환 후 음성 내용과 변환 결과를 비교


## **3. 미션2 : Summary**

* 세부사항
    * 문서요약 예제 파일을 참조하여 테스트 해 봅니다.
    * 코드를 참조하여, 원하는 형식에 맞게 요약이 되도록 프롬프트를 구성합니다.
        * 요약 시 중요 키워드들이 함께 도출되도록 합니다.
        * 가능하다면, 요약 문장 길이에 제한을 둡시다.
    * 반복문을 통해 요약하고, 결과를 데이터프레임에 추가합니다.
        * summary 열을 추가하고, 요약 결과를 입력
            * 요약결과와 키워드는 하나의 문자열로 붙여서 summary열에 추가

### (1) 문서 요약

* 문서 요약 예제

In [ ]:
input_text = '''
한국은행 총재가 "올해 성장률이 기존 전망치 2.4%보다 낮아질 가능성이 크다"며 "2.2∼2.3% 정도로 떨어지지 않을까 생각한다"고 밝혔습니다.
이 총재는 오늘(29일) 국회 기획재정위원회 국정감사에 출석해 한은의 전망을 크게 밑돈 3분기 성장률을 바탕으로 올해 성장률 전망치가 조정될 가능성에 대해 이렇게 말했습니다.
성장률 하락의 가장 큰 요인인 수출 감소의 배경에 대해 이 총재는 "금액 기준으로 봐서는 수출이 안 떨어졌는데, 수량을 기준으로 떨어졌다"며 "자동차 파업 등 일시적 요인과 화학제품·반도체의 중국과 경쟁 등으로 수량이 안 늘어나는 것 같은데, 원인을 더 분석해봐야 할 사안"이라고 진단했습니다.
다음 달 28일 열릴 기준금리 결정 방향에 대해서는 "금리 결정할 때 하나의 변수만 보지 않고 종합적으로 보는데, 우선 미국 대선과 연방준비제도 금리 결정으로 경제 상황이 어떻게 변할지 보겠다"고 밝혔습니다.
또 "아울러 이후 달러가 어떻게 될지, 수출 등 내년 경제 전망과 거시안전성 정책이 부동산·가계부채에 미치는 영향 등도 고려해 결정하겠다"고 말했습니다.
'''

system_role = '''당신은 신문기사에서 핵심을 요약하는 어시스턴트입니다.
응답은 다음의 형식을 지켜주세요
{"summary": \"텍스트 요약\"}
'''

response = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[
        {
            "role": "system",
            "content": system_role
        },
        {
            "role": "user",
            "content": input_text
        }
    ]
)

# 답변
answer = response.choices[0].message.content
print(answer)

* 문서 요약 함수로 생성

In [ ]:
def text_summary(input_text):
    # OpenAI 클라이언트 생성


    # 시스템 역할과 응답 형식 지정




    # 입력데이터를 GPT-3.5-turbo에 전달하고 답변 받아오기
    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
            {
                "role": "system",
                "content": system_role
            },
            {
                "role": "user",
                "content": input_text
            }
        ]
    )

    # 응답 받기


    # 응답형식을 정리하고 return



* 저장된 text를 하나씩 불러와서 요약하고 다시 저장하기

### (2) 전국 병원 응급실 정보 수집



#### 1) 인증키 발급

* 인증키 발급 절차
    * 1) data.go.kr 회원가입
    * 2) 국립중앙의료원_전국 응급의료기관 정보 조회 서비스
https://www.data.go.kr/data/15000563/openapi.do 로 이동
    * 3) 활용신청
        * 활용목적 : 기타(개인 학습 용도)
        * 상세 기능선택
            * 응급의료기관 목록정보 조회
            * 응급의료기관 위치정보 조회
            * 응급의료기관 기본정보 조회
    * 4) 인증키 확인
        * 마이페이지 > Open API > 활용신청현황
        * [승인] 국립중앙의료원_전국 응급의료기관 정보 조회 서비스
        * 일반 인증키(Decoding) 이용

#### 2) 데이터 수집

In [ ]:
# path 확인
path

In [ ]:
# 응급실 데이터 수집하기

url =
serviceKey =      # 여러분의 일반 인증키(Decoding)

params = {
    'serviceKey': serviceKey,
    'pageNo': '1', 'numOfRows': '1000',  # 전체 응급실 수가 500여개 됨. 1000개면 충분
    'format': 'xml'
}

response = requests.get(url, params = params)

# 정상 수행 되었다면 200
print(response)

In [ ]:
# response xml에서 주요 정보 찾기
root = ET.fromstring(response.text)

data = []

for item in root. findall('.//item'):
    duty_name = item.findtext('dutyName')
    duty_addr = item.findtext('dutyAddr')
    # 필요한 정보 추가


    # 빈 리스트 data에 딕시너리 형태({'칼럼이름':값, ...})로 저장(추가)


# 데이터프레임으로 변환



In [ ]:
# csv 파일로 저장(인덱스 제외)


## **Mission Complete!**

수고 많았습니다!